In [1]:
import pandas as pd
import os
rootdir = os.path.join(os.getcwd(), "..")

In [2]:
training_protocol ="joint"
horizon=5
trajectories=100
particles=10
mpc=5

In [3]:
template = "python run_pets_script.py --cs_seed {cs_seed} --search_space {search_space} --horizon {horizon} --num_random_trajectories {trajectories} --num_particles {particles} --mpc_seed {mpc} --apply_lookahead {apply_lookahead} --load_pretrained {load_pretrained};"
# for file in  scripts/sbatch/{search_space}/joint/{pretrained_dir}/cs_seed-{cs_seed}/test/horizon-{horizon}/trajectories-{trajectories}/particles-{particles}/MPC/mpc-{mpc}/*; do msub $file;done; "

subfolder1 = f"if [[ $load_pretrained  -eq  1 ]]; then subfolder1=pre-trained; else subfolder1=random-initialization; fi"
subfolder2 = f"if [[ $apply_lookahead  -eq  1 ]]; then subfolder2=LookAhead; else subfolder2=MPC; fi"

In [4]:
for search_space in ['4796', '5527', '5636', '5859', '5860',
                    '5891', '5906', '5965', '5970', '5971', '6766',
                    '6767', '6794', '7607', '7609', '5889']:
    script = "for particles in "
    for particles in [10, 100]:
        script += f"{particles} "
    script += "; do for horizon in "
    for horizon in [3 , 5, 10]:
        script += f"{horizon} "
    script += "; do for trajectories in "
    for trajectories in [10, 100]:
        script += f"{trajectories} "
    script += "; do for apply_lookahead in "
    for apply_lookahead in [0, 1]:
        script += f"{apply_lookahead} "
    script += "; do for load_pretrained in "
    for load_pretrained in [1, 0]:
        script += f"{load_pretrained} "

    try:
        results = pd.read_csv(os.path.join(rootdir, "collected", f"{search_space}-{training_protocol}.csv"), index_col=0).transpose()
        results["seeds"] = list(map(lambda x: int(x.replace("seed-","")), results.index.tolist()))
        cs_seed = int(results.sort_values(by="val_loss").iloc[0]["seeds"])

        script_prepare = script + f"; do python run_pets_script.py --cs_seed {cs_seed} --search_space {search_space} --horizon $horizon --num_random_trajectories $trajectories --num_particles $particles --mpc_seed {mpc} --apply_lookahead $apply_lookahead --load_pretrained $load_pretrained;"

        script_prepare += "done;done;done;done;done;"
        script_run = script + f"; do {subfolder1}; {subfolder2}; for file in  scripts/sbatch/{search_space}/joint/$subfolder1/cs_seed-{cs_seed}/test/horizon-$horizon/trajectories-$trajectories/particles-$particles/$subfolder2/mpc-{mpc}/*; do msub $file;done;"

        script_run += "done;done;done;done;done;"
        with open(os.path.join(rootdir, "scripts", "txt_preparation", f"prepare_scripts_for_inference_{search_space}.txt"), "w") as f:
            f.write(script_prepare)

        with open(os.path.join(rootdir, "scripts", "txt_run_inference",f"run_scripts_for_inference_{search_space}.txt"), "w") as f:
            f.write(script_run)

    except Exception as e:
        print(search_space)



5636
5859


In [5]:
script_run

'for particles in 10 100 ; do for horizon in 3 5 10 ; do for trajectories in 10 100 ; do for apply_lookahead in 0 1 ; do for load_pretrained in 1 0 ; do if [[ $load_pretrained  -eq  1 ]]; then subfolder1=pre-trained; else subfolder1=random-initialization; fi; if [[ $apply_lookahead  -eq  1 ]]; then subfolder2=LookAhead; else subfolder2=MPC; fi; for file in  scripts/sbatch/5889/joint/$subfolder1/cs_seed-45/test/horizon-$horizon/trajectories-$trajectories/particles-$particles/$subfolder2/mpc-5/*; do msub $file;done;done;done;done;done;done;'